# 목적
* 블로그 내용을 긁어와서 연관분석 진행
1. keyword관련 블로그 크롤링 
    정확도 기준으로 긁어오되 날짜도 같이 가져와서 나중 어떤 날짜에 집중되어 있는지 확인
    한 블로그당 단어빈도수 분석(필요할까)
2. 블로그 내 단어 토크나이즈하고 
3. 연관분석실시


In [3]:
# # 선행설치
# !pip install beautifulsoup4
# !pip install requests
# !pip install lxm
# # 연관분석
# !pip install apyori
# # 자연어처리
# !pip install konlpy (JDK 설치가 되어있어야함 )

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5984 sha256=5235bb621f4450c8a0ed214694dd194b7f3b0f6a4cf1cb568b9531188bd48e80
  Stored in directory: c:\users\try00\appdata\local\pip\cache\wheels\47\6f\0f\21a86f3679f7ed6bbe4dc6694f86818c5d85c2044bfab0f1e8
Successfully built apyori


In [2]:
from tqdm import tqdm_notebook  # 진행과정 시각화
from datetime import timedelta  # 시간날짜
from apyori import apriori  # 연관분석
import re
import json
import math
import datetime
import pandas as pd

import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup  # 크롤링

import konlpy
from konlpy.tag import Komoran  # 자연어처리
komoran = Komoran(userdic='./user_dictionary.txt')

In [2]:
#페이지 카운트 수로 api호출된다
#1,000개일때 3분 
#100*100개 만개의 경우 "비정상적인 트래픽을 감지"해서 오류걸림 
naver_client_id = "cGDVdveeg4egAgRCvXtA"
naver_client_secret = "pFnYuTsat0"
keyword = '양양'
display_count = 1
page_count = 1000
sort_type = 'sim'

In [3]:
def del_outword(string):
    #이모지제거
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)

    #분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
    han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d\u200b]')
    
    string = emoji_pattern.sub(r'',string)
    string = han.sub(r'',string)
    
    return string

In [300]:
def time_change(x):
    '''
    검색결과에 있는 문자를 날짜로 변경
    '''
    min_pattern = re.compile('[0-9]+'+"분 전")
    hour_pattern = re.compile('[0-9]+'+"시간 전")
    day_pattern = re.compile('[0-9]+'+"일 전")

    today = datetime.datetime.today().date()
    # 일자
    try:
        d = re.findall(day_pattern, x)[0][0]
        x = today - timedelta(days=int(d))
    except:
        pass
    # 시간
    try:
        d = re.findall(hour_pattern, x)[0]
        x = today
    except:
        pass
    # 분
    try:
        h = re.findall(min_pattern, x)[0]
        x = today
    except:
        pass

    if x == '어제':
        x = today - timedelta(days=1)

    elif type(x) == str:
        x = datetime.datetime.strptime(x.replace('.', '-')[:-1], '%Y-%m-%d')
    return x

In [29]:
# #네이버에서 검색했을때 url구조
# https://search.naver.com/search.naver
#     ?date_from=&
#     date_option=0&
#     date_to=&
#     dup_remove=1&
#     nso=&
#     post_blogurl=
#     &post_blogurl_without=
#     &query=%EC%96%91%EC%96%91
#     &sm=tab_pge
#     &srchby=all
#     &st=sim
#     &where=post
#     &start=11

In [ ]:
#sample 상세옵션없음
https://search.naver.com/search.naver?date_from=&date_option=0&date_to=&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=&query=%EC%96%91%EC%96%91&sm=tab_pge&srchby=all&st=sim&where=post&start=1
https://search.naver.com/search.naver?
    date_from=&
    date_option=0&
    date_to=&
    dup_remove=1&
    nso=&
    post_blogurl=&
    post_blogurl_without=&
    query=%EC%96%91%EC%96%91&
    sm=tab_pge&
    srchby=all&
    st=sim&
    where=post&
    start=1

date_from - YYYYMMDD : 시작날짜 date_option 8일때   
date_option - int : 0 : 전체, 2 : 1일, 3 : 1주, 4 : 1개월, 6 : 6개월, 7 : 1년, 8 : 기간지정  
date_to - YYYYMMDD : 마지막날짜 date_option 8일때  
dup_remove - int : 유사문서제거옵션 1 : 제거, 0 : 유지  
nso - str : _상세검색으로 추정_   
post_blogurl - ?  
post_blogurl_without - ?   
query - str : 검색어     
sm=tab_pge -?    
st - str : 정렬순서 sim : 유사도 date : 날짜    

In [21]:
keyword = "양양"
start_num = 1
end_num = 1001

In [462]:
#날짜수정가능하게 하는거 에러남
def naver_blog_crawling(keyword, start_num=1, end_num=101,date_option=0,date_from='',date_to=''):
    '''
    네이버 블로그 크롤링 함수
    네이버 블로그 검색결과를 크롤링하며, 1페이지당 10개씩을 검색한다
    
    keyword : string
     검색하고 싶은 키워드를 넣는다
    start_num : int (default = 1) 
     시작할 위치 1로 끝나는 단위 추천
    end_num : int (default = 101)
     끝나는 위치 1로 끝나는 단위 추천
    date_option : int (default = 0)
     주어지는 숫자에 의해 검색방법이 변경됨
     0 : 전체, 2 : 1일, 3 : 1주, 4 : 1개월, 6 : 6개월, 7 : 1년, 8 : 기간지정
    date_from : YYYYMMDD (default = "")
     date_option이 8일때 사용 검색 시작일자를 지정
    date_to : YYYMMDD (default = "")
     date_option이 8일때 사용 검색 마지막일자를 지정
    '''
    #url 찾는 패턴
    pattern =re.compile('href="'+'[A-z0-9\:\/\&\;\.\?\=]+')

    #저장위치
    postdates = []
    strings = []
    urls = []
    titles = []
    output_error = []
    connection_error = []
    count = 1

    #keyword와 시작넘버만 바꾸면서 진행하게끔
    base_url = 'https://search.naver.com/search.naver?date_from={date_from}&date_option={date_option}&date_to={date_to}&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=&query={keyword}&sm=tab_pge&srchby=all&st=sim&where=post&start={start}'

    #for문 돌려야하는 부분(각 검색결과의 시작은 1이고, 10개씩 보여짐,)
    for i in tqdm_notebook(range(start_num,end_num+1,10),desc = "page work"):
        search_list = base_url.format(keyword = keyword,start = i,
                                      date_option = date_option, date_from = date_from, date_to=date_to)
        response = requests.get(search_list)

        if response.status_code is 200:
            #1번 검색시 10개의 결과가 출력 따라서 section은 총 10개
            soup = BeautifulSoup(response.content,'lxml')
            sections = soup.findAll('li', attrs={'class':'sh_blog_top'})

            for section in sections:
                try:
                    #href부분만 가져오기 어려워서 정규표현식으로 검색
                    url = re.findall(pattern, str(section))[0].replace('?Redirect=Log&amp;logNo=','/').replace('href="','')
                    title = section.select_one('a.txt84').text
                    date = section.select_one('dd.txt_inline').text.strip()

                    #블로그 url안에 들어가기(아직 크롤링불가)
                    get_blog_post_content_code = requests.get(url)
                    get_blog_post_content_text = get_blog_post_content_code.text
                    get_blog_post_content_soup = BeautifulSoup(get_blog_post_content_text, 'lxml')

                    #크롤링가능한 url에 접속
                    real_blog_post_url = "http://blog.naver.com" + get_blog_post_content_soup.select('#mainFrame')[0].get('src')
                    get_real_blog_post_content_code = requests.get(real_blog_post_url)
                    get_real_blog_post_content_text = get_real_blog_post_content_code.text
                    get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'lxml')

                    # url (에러나면 위에서부터 에러남)
                    urls.append(real_blog_post_url)
                    #블로그명
                    titles.append(title)
                    #날짜
                    postdates.append(date)

                    #본문  (postviewarea 패턴과 se-main-container 2가지 유형이 있어 분리함) 
                    try:
                        blog_post_content = get_real_blog_post_content_soup.select('div#postViewArea')
                        if len(blog_post_content) == 0:
                            blog_post_content = get_real_blog_post_content_soup.select('div.se-main-container')

                        string = ""
                        for sentence in blog_post_content[0].stripped_strings:
                            string += " "+sentence.replace('\xa0'," ")
                        strings.append([string])
                        count += 1
                    except:
                        strings.append([np.NaN])
                        count += 1

                except Exception as ex: 
                    print('가져오기에러 {num}번째'.format(num = count),ex)
                    output_error.append(count)
                    count += 1
                    pass
            else:
                print('연결오류 {num}번째'.format(num = count),response.status_code)
                connection_error.append(count)
                count += 1
                
    crawling_df = pd.DataFrame({"post_dates":postdates, "title":titles, "full_text":strings, "url":urls})
    return crawling_df,output_error,connection_error
        

In [1]:
def naver_blog_crawling(keyword, start_num=1, end_num=101):
    '''
    네이버 블로그 크롤링 함수
    네이버 블로그 검색결과를 크롤링하며, 1페이지당 10개씩을 검색한다

    keyword : string
     검색하고 싶은 키워드를 넣는다
    start_num : int (default = 1) 
     시작할 위치 1로 끝나는 단위 추천
    end_num : int (default = 101)
     끝나는 위치 1로 끝나는 단위 추천
    date_option : int (default = 0)
     주어지는 숫자에 의해 검색방법이 변경됨
     0 : 전체, 2 : 1일, 3 : 1주, 4 : 1개월, 6 : 6개월, 7 : 1년, 8 : 기간지정
    date_from : YYYYMMDD (default = "")
     date_option이 8일때 사용 검색 시작일자를 지정
    date_to : YYYMMDD (default = "")
     date_option이 8일때 사용 검색 마지막일자를 지정
    '''
    # url 찾는 패턴
    pattern = re.compile('href="'+'[A-z0-9\:\/\&\;\.\?\=]+')

    # 저장위치
    postdates = []
    strings = []
    urls = []
    titles = []
    output_error = []
    connection_error = []
    count = 1

    # keyword와 시작넘버만 바꾸면서 진행하게끔
    base_url = 'https://search.naver.com/search.naver?date_from=&date_option=&date_to=&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=&query={keyword}&sm=tab_pge&srchby=all&st=sim&where=post&start={start}'

    # for문 돌려야하는 부분(각 검색결과의 시작은 1이고, 10개씩 보여짐,)
    for i in tqdm_notebook(range(start_num, end_num+1, 10), desc="page work"):
        search_list = base_url.format(keyword=keyword, start=i)
        response = requests.get(search_list)

        if response.status_code is 200:
            # 1번 검색시 10개의 결과가 출력 따라서 section은 총 10개
            soup = BeautifulSoup(response.content, 'lxml')
            sections = soup.findAll('li', attrs={'class': 'sh_blog_top'})

            for section in sections:
                try:
                    # href부분만 가져오기 어려워서 정규표현식으로 검색
                    url = re.findall(pattern, str(section))[0].replace(
                        '?Redirect=Log&amp;logNo=', '/').replace('href="', '')
                    title = section.select_one('a.txt84').text
                    date = section.select_one('dd.txt_inline').text.strip()

                    # 블로그 url안에 들어가기(아직 크롤링불가)
                    get_blog_post_content_code = requests.get(url)
                    get_blog_post_content_text = get_blog_post_content_code.text
                    get_blog_post_content_soup = BeautifulSoup(
                        get_blog_post_content_text, 'lxml')

                    # 크롤링가능한 url에 접속
                    real_blog_post_url = "http://blog.naver.com" + \
                        get_blog_post_content_soup.select('#mainFrame')[
                            0].get('src')
                    get_real_blog_post_content_code = requests.get(
                        real_blog_post_url)
                    get_real_blog_post_content_text = get_real_blog_post_content_code.text
                    get_real_blog_post_content_soup = BeautifulSoup(
                        get_real_blog_post_content_text, 'lxml')

                    # url (에러나면 위에서부터 에러남)
                    urls.append(real_blog_post_url)
                    # 블로그명
                    titles.append(title)
                    # 날짜
                    postdates.append(date)

                    # 본문  (postviewarea 패턴과 se-main-container 2가지 유형이 있어 분리함)
                    try:
                        blog_post_content = get_real_blog_post_content_soup.select(
                            'div#postViewArea')
                        if len(blog_post_content) == 0:
                            blog_post_content = get_real_blog_post_content_soup.select(
                                'div.se-main-container')

                        string = ""
                        for sentence in blog_post_content[0].stripped_strings:
                            string += " "+sentence.replace('\xa0', " ")
                        # 비언어 텍스트제거
                        string = del_outword(string)
                        strings.append([string])
                        count += 1
                    except:
                        strings.append(['X'])
                        count += 1

                except Exception as ex:
                    #print('가져오기에러 {num}번째'.format(num = count),ex)
                    output_error.append(count)
                    count += 1
                    pass
        else:
            #print('연결오류 {num}번째'.format(num = count),response.status_code)
            connection_error.append(count)
            count += 1

    out_length = len(output_error+connection_error)
    print("검색한 길이:", end_num-start_num+10)
    print("제외된 길이:", out_length)
    print("검색된 길이:", end_num-start_num+10-out_length)
    crawling_df = pd.DataFrame(
        {"post_dates": postdates, "title": titles, "full_text": strings, "url": urls})
#     crawling_df['post_dates'] = crawling_df['post_dates'].apply(lambda x : time_change(x))
    return crawling_df, output_error

In [252]:
keyword = "양양"
start_num = 1
end_num = 1001

In [253]:
crawling_df, output_error = naver_blog_crawling(keyword = keyword, start_num = start_num,end_num = end_num)

C:\Users\try00\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



검색한 길이: 1010
제외된 길이: 184
검색된 길이: 826


In [303]:
crawling_df['post_dates'] =crawling_df['post_dates'].apply(lambda x : time_change(x))

### df dt속성들
https://m.blog.naver.com/PostView.nhn?blogId=wideeyed&logNo=221603462366&proxyReferer=https%3A%2F%2Fwww.google.com%2F

In [325]:
crawling_df['post_dates'].dt.year.value_counts()

2020    461
2019    361
2018      4
Name: post_dates, dtype: int64

In [326]:
crawling_df['post_dates'].dt.month.value_counts()

3     180
1     155
2     119
12    106
11     88
10     58
8      24
7      24
9      23
4      20
6      19
5      10
Name: post_dates, dtype: int64

In [102]:
crawling_df.shape

(823, 4)

In [177]:
crawling_df.to_csv(keyword+"_"+str(end_num)+'.csv',encoding = 'utf-8')

### 검수용코드

In [14]:
#1개코드 
#저장위치
postdates = []
strings = []
urls = []
titles = []
output_error = []
connection_error = []
count = 1

pattern =re.compile('href="'+'[A-z0-9\:\/\&\;\.\?\=]+')
#keyword와 시작넘버만 바꾸면서 진행하게끔
base_url = 'https://search.naver.com/search.naver?date_from=&date_option=0&date_to=&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=&query={keyword}&sm=tab_pge&srchby=all&st=sim&where=post&start={start_num}'

#for문 돌려야하는 부분(각 검색결과의 시작은 1이고, 10개씩 보여짐,)
search_list = base_url.format(keyword = "양양",start_num = 11)
response = requests.get(search_list)

if response.status_code is 200:
    #1번 검색시 10개의 결과가 출력 따라서 section은 총 10개
    soup = BeautifulSoup(response.content,'lxml')
    sections = soup.findAll('li', attrs={'class':'sh_blog_top'})
    section = sections[0]
    
    #href부분만 가져오기 어려워서 정규표현식으로 검색
    url = re.findall(pattern, str(section))[0].replace('?Redirect=Log&amp;logNo=','/').replace('href="','')
    title = section.select_one('a.txt84').text
    date = section.select_one('dd.txt_inline').text.strip()

    #블로그 url안에 들어가기(아직 크롤링불가)
    get_blog_post_content_code = requests.get(url)
    get_blog_post_content_text = get_blog_post_content_code.text
    get_blog_post_content_soup = BeautifulSoup(get_blog_post_content_text, 'lxml')

    #크롤링가능한 url에 접속
    real_blog_post_url = "http://blog.naver.com" + get_blog_post_content_soup.select('#mainFrame')[0].get('src')
    get_real_blog_post_content_code = requests.get(real_blog_post_url)
    get_real_blog_post_content_text = get_real_blog_post_content_code.text
    get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'lxml')

    # url (에러나면 위에서부터 에러남)
    urls.append(real_blog_post_url)
    #블로그명
    titles.append(title)
    #날짜
    postdates.append(date)

    #본문  (postviewarea 패턴과 se-main-container 2가지 유형이 있어 분리함) 
    try:
        blog_post_content = get_real_blog_post_content_soup.select('div#postViewArea')
        if len(blog_post_content) == 0:
            blog_post_content = get_real_blog_post_content_soup.select('div.se-main-container')

        string = ""
        for sentence in blog_post_content[0].stripped_strings:
            string += " "+sentence.replace('\xa0'," ")
            
        strings.append([string])
        count += 1
    except:
        strings.append([np.NaN])
        count += 1


# crawling_df = pd.DataFrame({"post_dates":postdates, "title":titles, "full_text":strings, "url":urls})


In [19]:
keyword = "양양"
start_num = 1
end_num = 11

In [24]:
for i in tqdm_notebook(range(start_num,end_num+1,10),desc = "page work"):
    print(i)

C:\Users\try00\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


1
11



In [26]:
search_list

'https://search.naver.com/search.naver?date_from=&date_option=&date_to=&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=&query=양양&sm=tab_pge&srchby=all&st=sim&where=post&start=11'

### 함수화이전 

In [42]:
searched_urls = []
for section in sections:
    url = re.findall(pattern, str(section))[0].replace('?Redirect=Log&amp;logNo=','/').replace('href="','')
    searched_urls.append(url)

In [69]:
#url 찾는 패턴
pattern =re.compile('href="'+'[A-z0-9\:\/\&\;\.\?\=]+')

#저장위치
postdates = []
strings = []
urls = []
titles = []
output_error = []
connection_error = []
count = 1

#keyword와 시작넘버만 바꾸면서 진행하게끔
base_url = 'https://search.naver.com/search.naver?date_from=&date_option=&date_to=&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=&query={keyword}&sm=tab_pge&srchby=all&st=sim&where=post&start={start}'

#for문 돌려야하는 부분(각 검색결과의 시작은 1이고, 10개씩 보여짐,)
for i in tqdm_notebook(range(start_num,end_num+1,10),desc = "page work"):
    search_list = base_url.format(keyword = keyword,start = i)
    response = requests.get(search_list)

    if response.status_code == 200:
        #1번 검색시 10개의 결과가 출력 따라서 section은 총 10개
        soup = BeautifulSoup(response.content,'lxml')
        sections = soup.findAll('li', attrs={'class':'sh_blog_top'})

        for section in sections:
            try:
                #href부분만 가져오기 어려워서 정규표현식으로 검색
                url = re.findall(pattern, str(section))[0].replace('?Redirect=Log&amp;logNo=','/').replace('href="','')
                title = section.select_one('a.txt84').text
                date = section.select_one('dd.txt_inline').text.strip()

                #블로그 url안에 들어가기(아직 크롤링불가)
                get_blog_post_content_code = requests.get(url)
                get_blog_post_content_text = get_blog_post_content_code.text
                get_blog_post_content_soup = BeautifulSoup(get_blog_post_content_text, 'lxml')

                #크롤링가능한 url에 접속
                real_blog_post_url = "http://blog.naver.com" + get_blog_post_content_soup.select('#mainFrame')[0].get('src')
                get_real_blog_post_content_code = requests.get(real_blog_post_url)
                get_real_blog_post_content_text = get_real_blog_post_content_code.text
                get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'lxml')

                # url (에러나면 위에서부터 에러남)
                urls.append(real_blog_post_url)
                #블로그명
                titles.append(title)
                #날짜
                postdates.append(date)

                #본문  (postviewarea 패턴과 se-main-container 2가지 유형이 있어 분리함) 
                try:
                    blog_post_content = get_real_blog_post_content_soup.select('div#postViewArea')
                    if len(blog_post_content) == 0:
                        blog_post_content = get_real_blog_post_content_soup.select('div.se-main-container')

                    string = ""
                    for sentence in blog_post_content[0].stripped_strings:
                        string += " "+sentence.replace('\xa0'," ")
                    #비언어 텍스트제거 
                    string = del_outword(string)
                    strings.append([string]) 
                    count += 1
                except:
                    strings.append([np.NaN])
                    count += 1
                    pass

            except Exception as ex: 
                print('가져오기에러 {num}번째'.format(num = count),ex)
                output_error.append(count)
                count += 1
                pass
    else:
        print('연결오류 {num}번째'.format(num = count),response.status_code)
        connection_error.append(count)
        count += 1

C:\Users\try00\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


가져오기에러 1번째 list index out of range
가져오기에러 2번째 list index out of range
가져오기에러 5번째 list index out of range
가져오기에러 7번째 list index out of range
가져오기에러 8번째 list index out of range
가져오기에러 13번째 list index out of range
가져오기에러 18번째 list index out of range



### 2020-04-01 17:04:35 길이가 다른것에 해결책
* 에러나면 NAN추가해버리기

In [40]:
#이전코드
def get_blog_post(keyword, display_count, page_count, sort_type,save = True):
    '''
    keyword : 검색하고 싶은 키워드
    display_count : 한 페이지당 표출할 개수 min = 10, max = 100
    page_count : 총 진행할 페이지의 수 min = 1, max = 1000
    sort_type : 정렬옵션 "sim" (유사도순), "date" (날짜순)
    
    display_count는 표시되는 개수에 불과하며 page_count가 1개씩 넘어갈때마다 나머지 diplay_count가 중복됨
    ''' 
    #날짜와 본문내용 저장
    postdates = []
    strings = []
    urls = []
    titles = []
    blogger_names = []
    responses = []

    encode_keyword = urllib.parse.quote(keyword)
    # get_blog_search_result_pagination_count로 처리할 수있는 페이지수를 1부터 페이지수까지 까지 각각 하나씩 긁어옴 
    for i in tqdm_notebook(range(1, page_count + 1),desc = "page work"):
        url = "https://openapi.naver.com/v1/search/blog?query=" + encode_keyword + "&display=" + str(
            display_count) + "&start=" + str(i) + "&sort=" + sort_type

        request = urllib.request.Request(url)

        request.add_header("X-Naver-Client-Id", naver_client_id)
        request.add_header("X-Naver-Client-Secret", naver_client_secret)

        #접속오류났을때 종료후 저장
        try:
            response = urllib.request.urlopen(request)
            response_code = response.getcode()

            # respon결과가 200일때 결과읽어옴 
            if response_code is 200:
                response_body = response.read()
                response_body_dict = json.loads(response_body.decode('utf-8'))
                responses.append(response_body_dict)
                try:
                #items의 개수만큼씩 진행
                    for j in range(0, len(response_body_dict['items'])):
                        blog_post_url = response_body_dict['items'][j]['link'].replace("amp;", "")

                        #블로그 url안에 들어가기(아직 크롤링불가)
                        get_blog_post_content_code = requests.get(blog_post_url)
                        get_blog_post_content_text = get_blog_post_content_code.text
                        get_blog_post_content_soup = BeautifulSoup(get_blog_post_content_text, 'lxml')
                        #크롤링가능한 url에 접속
                        real_blog_post_url = "http://blog.naver.com" + get_blog_post_content_soup.select('#mainFrame')[0].get('src')
                        get_real_blog_post_content_code = requests.get(real_blog_post_url)
                        get_real_blog_post_content_text = get_real_blog_post_content_code.text
                        get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'lxml')
                        #본문부분 추출 
                        blog_post_content = get_real_blog_post_content_soup.select('div#postViewArea')
                        if len(blog_post_content) == 0:
                            blog_post_content = get_real_blog_post_content_soup.select('div.se-main-container')

                        #포스트날짜
                        postdate = datetime.datetime.strptime(response_body_dict['items'][j]['postdate'],"%Y%m%d").strftime("%y.%m.%d")
                        postdates.append(postdate)

                        #제목
                        remove_html_tag = re.compile('<.*?>')
                        title = re.sub(remove_html_tag, '', response_body_dict['items'][j]['title'])
                        titles.append(title)

                        #블로거이름
                        blogger_name = response_body_dict['items'][j]['bloggername']
                        blogger_names.append(blogger_name)

                        #전체 텍스트 
                        string = ""
                        for sentence in blog_post_content[0].stripped_strings:
                            string += " "+sentence.replace('\xa0'," ")
                        #비언어 텍스트제거 
                        string = del_outword(string)
                        strings.append([string]) 

                        #url
                        urls.append(real_blog_post_url)

                except Exception as ex:
                    print('추출에러 {i}번째'.format(i),ex)
                    pass
            else :
                pass

        except Exception as ex:
            print('open에러 {i}번째'.format(i),ex)
            pass
                
    # utf-8형식으로 저장하면 엑셀에서 열때 에러나지만, load는 가능
    if save == True:
        crawling_df = pd.DataFrame({"post_dates":postdates, "title":titles, "blogger_name":blogger_names, "full_text":strings, "url":urls})
        crawling_df.to_csv(keyword+"_"+str(display_count* page_count)+".csv",encoding='utf-8',index= False)
    return crawling_df

In [41]:
crawling_df = get_blog_post(keyword, display_count, page_count, sort_type)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


ValueError: arrays must all be same length

### 연관분석

In [103]:
# crawling_df_s = pd.read_csv('양양_1000.csv',encoding='utf-8')
crawling_df_s['title'].nunique()

NameError: name 'crawling_df_s' is not defined

In [104]:
komoran = Komoran(userdic='user_dictionary.txt')

#이모티콘 제거 [출처](https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python)

In [105]:
emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)

#분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d]')

In [168]:
sw = ['오늘','시간','최근','지난','관련','이번','이날','현재','10','올해','지난해','있다','이상','기준','때문',
     "메뉴","주문","강원도",'정도','도착','생각','양양군','사진','위치']
all_nouns =[]

for i in tqdm_notebook(range(0,len(crawling_df))):
    try:
        nouns = komoran.nouns(crawling_df['full_text'][i][0])
    except :
        nouns = crawling_df['full_text'][i][0].replace('\u200b'," ")
        nouns = emoji_pattern.sub(r'',nouns)
        nouns = han.sub(r'',TEXT_refine)
    clean_nouns = list(filter(lambda word : (len(word)>1)&(word not in sw),nouns))
    all_nouns.append(clean_nouns)

C:\Users\try00\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [190]:
crawling_df.iloc[411]['full_text']

[' 동절기 숙소 당첨으로 다녀온 양양 주말엔 예약하기 쉽지 않던 양양솔비치 버스타고 혼자 잘 다녀왔다 처음으로 국내 장거리 뚜벅이여행 을 했는데 할만하다 + 세상못할짓이다 모두를 경험했다 어쨌거나 숙소 후기부터 시작하자 이게 호텔이고 주변으로 빌라형이 있었음 양양솔비치는 예전에 있던 숙소를 개조()한 거라 낡은 느낌이 많다고 하는 후기가 많았는데 뭐 그냥 적당했다 막 낡아서 서운할정도는 아니다 이런얘기를 못듣고 갔으면 눈치챌 여력도 없었을듯  원래 온돌룸으로 예약했는데(혼자갈 생각아니었음) 혼자간김에 침대룸으로 바꾸었다 다행히 가능 리셉션에 있던 직원들은 친절했다 여기는 예약을 해두면 오는 순서대로 같은 레벨 중 좋은()방을 배정해주는 듯 했다 그러니 체크인 시간 맞춰서 빨리 후닥 가십시다 더블싱글 침대 두개 혼자쓰기 산전망이라서 베란다 나가면 이런뷰 보입니다 방은 작다 암막커튼을 걷지 않으면 갑갑하다 출장지에서 슈페리어 룸으로 많이들 가는데 비교해보면 역시 작다 비교하기 나름이라지만 양양이 싱가폴이나 런던은 아니잖;; 온돌방이 방사이즈가 같다는데 정돈 안된 갑갑함을 경험할뻔 했다 바꾸길 잘했다 호텔이라서 국내에서 자주가던 콘도와 다르다 클린형이라더니 취사도구가 전혀없다 그냥 해외 호텔 생각하면 된다 전기포트는 있다 각종 잔도 다 있고 아주 비싼 미니바는 전세계 공통 이 정보를 미리 알았기에 미리 시내에서 내가 먹을 맥주는 사가지고 들어갔다 내부 매점도 당연히 비쌈 각종 술잔 구비 작은 미니바용 냉장고 있고요 미니바 가격 캔콜라 하나 3천원은너무하잖니 화장대 하나 있고 드라이기가 있긴한데 머리 말리다 인내심 테스트 당하는줄 알고 두리번 이방만 고장났었던진 모르겠으나 찬바람 약한뜨거운바람까지 나오고 강한 뜨거운바람 불가한 드라이기망가진걸까대체 어떻게 화장실도 깔끔하다 어메니티는 Aveda 민트샴푸가 시원했다 수압이 좀 약해서 별로였는데 욕실 구배가 잘못된듯 샤워하고 나왔더니 욕실밖으로 물이 많이 넘쳤()는데 하마터면 방쪽 카펫으로 물 넘칠뻔 했다 샴푸든 수건이

In [210]:
import datetime
from datetime import timedelta

In [211]:
today = datetime.datetime.today().date()

In [214]:
today - timedelta(days=1)

datetime.date(2020, 4, 1)

In [247]:
crawling_df['post_dates'] = crawling_df['post_dates'].apply(lambda x : time_change(x))

In [250]:
crawling_df['post_dates'].value_counts()

2020-03-07    765
2020-03-31     19
2020-04-01     10
2020-03-26      8
2020-03-29      7
2020-03-27      6
2020-03-30      5
2020-03-28      4
Name: post_dates, dtype: int64

In [194]:
#all_noun unlist
aa = [y for x in all_nouns for y in x]

In [198]:
pd.Series(aa).value_counts().head(20)

양양     5547
바다     1496
여행     1310
서핑     1005
해변      736
맛집      725
낙산사     723
카페      722
속초      713
사람      695
비치      573
쏠비치     566
호텔      521
느낌      515
겨울      492
송이      467
조대      451
모습      430
연어      426
마음      424
dtype: int64

In [170]:
noun_df = pd.DataFrame({'nouns':all_nouns})

In [171]:
rules = apriori(all_nouns, min_support=0.2, min_condience =0.3)

In [172]:
results  = list(rules)

In [173]:
results_df = pd.DataFrame(columns =["lhs",'rhs','support','confidence','lift'])
index =0
for row in results:   
    support = row[1]
    ordered_stat = row[2]
    for ordered_item in ordered_stat:
        lhs = " ".join(x for x in ordered_item[0])
        rhs = " ".join(x for x in ordered_item[1])
        confidence = ordered_item[2]
        lift = ordered_item[3]
        results_df.loc[index] = [lhs,rhs,support,confidence,lift]
        index = index +1

In [174]:
results_df = results_df.sort_values(by="confidence",ascending=False)

In [175]:
results_df

,lhs,rhs,support,confidence,lift
64,모습,양양,0.241505,1.000000,1.011043
43,날씨,양양,0.228155,1.000000,1.011043
116,커피,양양,0.201456,1.000000,1.011043
61,맛집,양양,0.259709,1.000000,1.011043
125,여행 느낌,양양,0.201456,1.000000,1.011043
...,...,...,...,...,...
2,,거리,0.202670,0.202670,1.000000
48,,여행 느낌,0.201456,0.201456,1.000000
120,,양양 여행 느낌,0.201456,0.201456,1.000000
25,,커피,0.201456,0.201456,1.000000
